In [1]:
import numpy as np
import gurobipy as gp
from functions import *
from data_store import *
from Class_UAV import *

In [2]:
# Defining parameters
pi = 100 # Penalty for unmet demand
T = 1 # Time horizon 1 hour  
e = 4 # maximal number of open depots
p = 10 # maximal number of open launch points
Q_T = 2000 # capacity of the vehicle
c_SD = 10 # cost for using a small drone
c_LD = 20 # cost for using a large drone
v_T = 45 # speed of the vehicle: 45 km/hr
tau = 2 # setup time for small drone to deliver: 2 miniutes
K_SD = 30 # maximal number of small drones
K_LD = 5 # maximal number of large drones

num_candidates_depots = 10
num_candidates_launch_points = 25
num_gathering_points = 40

In [3]:
S = 12

for s in range(S):
    gp_demand_and_location, D_location, LP_location = generate_all_locations(num_GP=num_gathering_points , cand_D=num_candidates_depots , cand_LP=num_candidates_launch_points)
    gp_demand = gp_demand_and_location[0]
    gp_location = gp_demand_and_location[1]
    save_to_csv("GP", s + 1, gp_demand, gp_location)
    save_to_csv("DP", s + 1, None, D_location)
    save_to_csv("LP", s + 1, None, LP_location)
    

Files already exist. The function will not run.
Files already exist. The function will not run.
Files already exist. The function will not run.
Files already exist. The function will not run.
Files already exist. The function will not run.
Files already exist. The function will not run.
Files already exist. The function will not run.
Files already exist. The function will not run.
Files already exist. The function will not run.
Files already exist. The function will not run.
Files already exist. The function will not run.
Files already exist. The function will not run.
Files already exist. The function will not run.
Files already exist. The function will not run.
Files already exist. The function will not run.
Files already exist. The function will not run.
Files already exist. The function will not run.
Files already exist. The function will not run.
Files already exist. The function will not run.
Files already exist. The function will not run.
Files already exist. The function will n

In [4]:
demand_list , GP_location_list = read_csv_files("GP", S)
DP_location_list = read_csv_files("DP", S)
LP_location_list = read_csv_files("LP", S)


Data arrays read complete
Data arrays read complete
Data arrays read complete


In [6]:
smalldrones = SmallDrone("SD")
largedrones = LargeDrone("LD")

Q_SD = smalldrones.max_payload
Q_LD = largedrones.max_payload

#### SDA Part

In [ ]:
# Initializations
UB = np.inf
LB = -np.inf
tolerance = 1e-6

R = []

# Create a new model
m = gp.Model("scenario_decomposition")
# Add first stage variables
u = m.addVars(num_candidates_depots, vtype=gp.GRB.BINARY, name="depot")
v = m.addVars(num_candidates_launch_points, vtype=gp.GRB.BINARY, name="launch_point")
w = m.addVars(num_candidates_depots , num_candidates_launch_points, vtype=gp.GRB.BINARY, name="LP_to_DP")
# Add second stage variables
g_SD = m.addVars(K_SD, vtype=gp.GRB.BINARY, name="small_drone_usage")
g_LD = m.addVars(K_LD, vtype=gp.GRB.BINARY, name="large_drone_usage")
x    = m.addVars(num_gathering_points, num_candidates_launch_points + num_candidates_launch_points, K_SD, vtype=gp.GRB.BINARY, name="small_drone_assignment")
y    = m.addVars(num_gathering_points, num_candidates_depots, K_LD, vtype=gp.GRB.BINARY, name="large_drone_assignment")
o    = m.addVars(num_gathering_points, vtype=gp.GRB.CONTINUOUS, name="unmet_demand")

while True:
    # Solve the optimization problem for each scenario
    for s in range(S):
        q = demand_list[s] # withdraw current demand
        GP_location = GP_location_list[s] # withdraw current gathering point locations
        D_location = DP_location_list[s] # withdraw current depot locations
        LP_location = LP_location_list[s] # withdraw current launch point locations

        # Add objective function
        m.setObjective( gp.quicksum(pi * o[i] for i in range(num_gathering_points)) + 
                        gp.quicksum(c_SD * g_SD[k] for k in range(K_SD)) + 
                        gp.quicksum(c_LD * g_LD[k] for k in range(K_LD)) ,
                       gp.GRB.MINIMIZE)
        # Add constraints
        m.addConstr(gp.quicksum(u[d] for d in range(num_candidates_depots)) <= e)
        m.addConstr(gp.quicksum(v[l] for l in range(num_candidates_launch_points)) <= p)
        for l in range(num_candidates_launch_points):
            m.addConstr(gp.quicksum(w[d,l] for d in range(num_candidates_depots)) == v[l])
            m.addConstr(gp.quicksum( q[i] * x[i,l,k] for i in range(num_gathering_points) for k in range(K_SD)) <= Q_T * v[l] )

        for k in range(K_SD):
            m.addConstr(gp.quicksum(x[i,l,k] for i in range(num_gathering_points) for l in range(num_candidates_launch_points)) <= 1)

        for i in range(num_gathering_points):
            m.addConstr( q[i] 
                        -  Q_SD * gp.quicksum( x[i,l,k] for l in range(num_candidates_depots+num_candidates_launch_points) for k in range(K_SD)) 
                        -  Q_LD * gp.quicksum( y[i,d,k] for d in range(num_candidates_depots) for k in range(K_SD+K_LD))
                        <= o[i])
            for d in range(num_candidates_depots):
                for k in range( K_SD+K_LD ):
                    m.addConstr(y[i,d,k] <= u[d])
            for l in range(num_candidates_launch_points):
                for k in range(K_SD):
                    m.addConstr(x[i,l,k] <= v[l])
        # Constraints for Speed
        # How? 
        # Also need to calculate the distance after the earthquake
    # Check convergence
    if UB - LB < tolerance:
        break